# a faire
* pays de paul 
    * regarder du coté d'open street map car je pense meilleur truc même si va prendre du temps : https://nominatim.org/release-docs/develop/api/Search/
* extraction SUBJECT des fois sur 2 lignes (\n)

In [168]:
import re
import spacy
from collections import Counter
from string import punctuation
from os import listdir
import time

current_year = time.strftime("%y", time.localtime())

nlp = spacy.load('en_core_web_lg')
nlp.vocab["CLASSIFIED"].is_stop = True # On ajoute CLASSIFIED au stop words

In [21]:
import pprint

In [2]:
data_path = './echantillon-cablegate_converted/'
files = [f for f in listdir(data_path) if f[-4:] == '.txt']

In [170]:
# Cleaning
res = {}
for i in range(0, 4):
    with open(data_path + files[i], encoding="utf8", errors='ignore') as f:
        content = f.read()

        extracted_doc = {}
        
        extracted_doc['doc_name'] = files[i][:-4]
        #extracted_doc['date'] = extract_date(content)
        extracted_doc['tags'] = extract_tags(content)
        extracted_doc['from'] = extract_from(content)
        extracted_doc['place_of_document'] = re.findall(r"[A-Z]+", extracted_doc['doc_name'])[-1] # -1 because we take the last one
        extracted_doc['subject'] = extract_subject(content)
        extracted_doc['most_common_words'] = extract_most_common_words(content)
        extracted_doc['keywords'] = extract_keywords(content)

        ner = dict([(str(x), x.label_) for x in nlp(content).ents])
        extracted_doc['people_involved'] = extract_people_involved(ner)
        extracted_doc['place_involved'] = extract_place_involved(ner)
        extracted_doc['entity_involved'] = extract_entity_involved(ner)


        res[i] = extracted_doc

In [171]:
pprint.pprint(res)

{0: {'doc_name': '06HONGKONG4795',
     'entity_involved': ['RHMFIUU/HQ',
                         'USDOC',
                         'OEA',
                         'LHINES/DFARROW USDOC',
                         'FCS',
                         'the Export Administration Act',
                         'the Office of Enforcement Analysis',
                         'the USDOC Bureau of Industry and Security',
                         'BIS',
                         'Export Control',
                         'Advanced Energy-Shenzhen ',
                         'Baltrans',
                         'ECCN',
                         'International Rectifier of Leominster',
                         'International Rectifier',
                         'Advanced Energy',
                         'ECO',
                         'Airfreight Operations',
                         'Operations Manager',
                         'Airfreight',
                         'Federal Express',
               

In [16]:
def extract_date(content):
    print(type(content))
    date_line = re.search(r"[A-Z]\s[0-9]{6}[A-Z]\s[A-Z]{3}\s[0-9]{2}", content)
    
    date_line = date_line.group()
    date_month = date_line.split(" ")[2]
    date_year = date_line.split(" ")[3]
    
    months = ['JAN', 'FEV', 'MAR', "APR", "MAY", "JUN", "JUl", "AUG", "SEP", "OCT", "NOV", "DEC"]
    
    date = {}
    date['year'] = None
    date['month'] = None
    if date_month in months :
        if int(date_year) > int(current_year):
            year = "19"+date_year
        else:
            year="20"+date_year
        date['year'] = year
        date['month'] = date_month
    else :
        print("ERR : unexpected month =", date_months)
    return date

In [26]:
def extract_tags(content):
    tags_list = [line for line in content.splitlines() if line.startswith(r"TAGS:")]
    return tags_list[0].split()[1:]

In [96]:
def extract_from(content):
    #fm_list = [line for line in content.splitlines() if line.startswith("FM ")]
    #return fm_list[0][3:-1] # y a toujours un espace à la fin
    FROM = re.search(r"\nFM\s[A-Z\s]+\nTO ", content)
    if FROM != None :
        FROM = FROM.group()[4:-4]
    else:
        #print(re.search(r"\nFOR\s[A-Z\/s]+", content))
        FROM = re.search(r"\nFOR\s[A-Z\s\/]+\n", content)
        if FROM != None :
            FROM = FROM.group()[5:-1]
            FROM =re.split(', | AND ',FROM)
    return FROM

In [11]:
def extract_subject(content):
    subject_list = [line for line in content.splitlines() if line.startswith("SUBJECT: ")]
    return subject_list[0][9:-1]# y a toujours un espace à la fin

In [154]:
def extract_most_common_words(content):
    doc = nlp(content)
    # all tokens that arent stop words or punctuations
    words = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text not in ("\n", "\n \n", "\n \n \n")]

    # fiftenth most common tokens
    word_freq = Counter(words)
    common_words = word_freq.most_common(15)
    return [word for word in common_words if len(word[0])>1]

In [158]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.is_stop or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

In [14]:
def extract_keywords(content):
    output = set(get_hotwords(content))
    return [x[0] for x in Counter(output).most_common(5)]

In [138]:
def extract_entity_involved(ner):
    entity_involved = [key.replace('\n', '').replace('   ', ' ').replace('  ',' ') for key in ner.keys() if ner[key] == 'ORG' or ner[key] == 'NORP' ]
    res = []
    for entity in entity_involved: # to remove duplicates
        if entity not in res:
            res.append(entity)
    return res 

In [139]:
def extract_place_involved(ner):
    place_involved = [key.replace('\n', '').replace('   ', ' ').replace('  ', ' ') for key in ner.keys() if ner[key] == 'GPE' or ner[key] == 'LOC' ]
    res = []
    for place in place_involved: # to remove duplicates
        if place not in res:
            res.append(place)
    return res 

In [169]:
def extract_people_involved(ner): # on laisse sipdis parce que peut être une personne
    people_involved = [key.replace('\n', '').replace('   ', ' ').replace('  ', ' ') for key in ner.keys() if ner[key] == 'PERSON']
    res = []
    for people in people_involved: # to remove duplicates
        if people not in res:
            res.append(people)
    return res 

In [173]:
from geopy import geocoders

gn = geocoders.Google()
place, (lat, lng) = gn.geocode("istanbul")
print(place)

AttributeError: module 'geopy.geocoders' has no attribute 'Google'

In [174]:
Country = [
    ('US', 'United States'),
    ('AF', 'Afghanistan'),
    ('AL', 'Albania'),
    ('DZ', 'Algeria'),
    ('AS', 'American Samoa'),
    ('AD', 'Andorra'),
    ('AO', 'Angola'),
    ('AI', 'Anguilla'),
    ('AQ', 'Antarctica'),
    ('AG', 'Antigua And Barbuda'),
    ('AR', 'Argentina'),
    ('AM', 'Armenia'),
    ('AW', 'Aruba'),
    ('AU', 'Australia'),
    ('AT', 'Austria'),
    ('AZ', 'Azerbaijan'),
    ('BS', 'Bahamas'),
    ('BH', 'Bahrain'),
    ('BD', 'Bangladesh'),
    ('BB', 'Barbados'),
    ('BY', 'Belarus'),
    ('BE', 'Belgium'),
    ('BZ', 'Belize'),
    ('BJ', 'Benin'),
    ('BM', 'Bermuda'),
    ('BT', 'Bhutan'),
    ('BO', 'Bolivia'),
    ('BA', 'Bosnia And Herzegowina'),
    ('BW', 'Botswana'),
    ('BV', 'Bouvet Island'),
    ('BR', 'Brazil'),
    ('BN', 'Brunei Darussalam'),
    ('BG', 'Bulgaria'),
    ('BF', 'Burkina Faso'),
    ('BI', 'Burundi'),
    ('KH', 'Cambodia'),
    ('CM', 'Cameroon'),
    ('CA', 'Canada'),
    ('CV', 'Cape Verde'),
    ('KY', 'Cayman Islands'),
    ('CF', 'Central African Rep'),
    ('TD', 'Chad'),
    ('CL', 'Chile'),
    ('CN', 'China'),
    ('CX', 'Christmas Island'),
    ('CC', 'Cocos Islands'),
    ('CO', 'Colombia'),
    ('KM', 'Comoros'),
    ('CG', 'Congo'),
    ('CK', 'Cook Islands'),
    ('CR', 'Costa Rica'),
    ('CI', 'Cote D`ivoire'),
    ('HR', 'Croatia'),
    ('CU', 'Cuba'),
    ('CY', 'Cyprus'),
    ('CZ', 'Czech Republic'),
    ('DK', 'Denmark'),
    ('DJ', 'Djibouti'),
    ('DM', 'Dominica'),
    ('DO', 'Dominican Republic'),
    ('TP', 'East Timor'),
    ('EC', 'Ecuador'),
    ('EG', 'Egypt'),
    ('SV', 'El Salvador'),
    ('GQ', 'Equatorial Guinea'),
    ('ER', 'Eritrea'),
    ('EE', 'Estonia'),
    ('ET', 'Ethiopia'),
    ('FK', 'Falkland Islands (Malvinas)'),
    ('FO', 'Faroe Islands'),
    ('FJ', 'Fiji'),
    ('FI', 'Finland'),
    ('FR', 'France'),
    ('GF', 'French Guiana'),
    ('PF', 'French Polynesia'),
    ('TF', 'French S. Territories'),
    ('GA', 'Gabon'),
    ('GM', 'Gambia'),
    ('GE', 'Georgia'),
    ('DE', 'Germany'),
    ('GH', 'Ghana'),
    ('GI', 'Gibraltar'),
    ('GR', 'Greece'),
    ('GL', 'Greenland'),
    ('GD', 'Grenada'),
    ('GP', 'Guadeloupe'),
    ('GU', 'Guam'),
    ('GT', 'Guatemala'),
    ('GN', 'Guinea'),
    ('GW', 'Guinea-bissau'),
    ('GY', 'Guyana'),
    ('HT', 'Haiti'),
    ('HN', 'Honduras'),
    ('HK', 'Hong Kong'),
    ('HU', 'Hungary'),
    ('IS', 'Iceland'),
    ('IN', 'India'),
    ('ID', 'Indonesia'),
    ('IR', 'Iran'),
    ('IQ', 'Iraq'),
    ('IE', 'Ireland'),
    ('IL', 'Israel'),
    ('IT', 'Italy'),
    ('JM', 'Jamaica'),
    ('JP', 'Japan'),
    ('JO', 'Jordan'),
    ('KZ', 'Kazakhstan'),
    ('KE', 'Kenya'),
    ('KI', 'Kiribati'),
    ('KP', 'Korea (North)'),
    ('KR', 'Korea (South)'),
    ('KW', 'Kuwait'),
    ('KG', 'Kyrgyzstan'),
    ('LA', 'Laos'),
    ('LV', 'Latvia'),
    ('LB', 'Lebanon'),
    ('LS', 'Lesotho'),
    ('LR', 'Liberia'),
    ('LY', 'Libya'),
    ('LI', 'Liechtenstein'),
    ('LT', 'Lithuania'),
    ('LU', 'Luxembourg'),
    ('MO', 'Macau'),
    ('MK', 'Macedonia'),
    ('MG', 'Madagascar'),
    ('MW', 'Malawi'),
    ('MY', 'Malaysia'),
    ('MV', 'Maldives'),
    ('ML', 'Mali'),
    ('MT', 'Malta'),
    ('MH', 'Marshall Islands'),
    ('MQ', 'Martinique'),
    ('MR', 'Mauritania'),
    ('MU', 'Mauritius'),
    ('YT', 'Mayotte'),
    ('MX', 'Mexico'),
    ('FM', 'Micronesia'),
    ('MD', 'Moldova'),
    ('MC', 'Monaco'),
    ('MN', 'Mongolia'),
    ('MS', 'Montserrat'),
    ('MA', 'Morocco'),
    ('MZ', 'Mozambique'),
    ('MM', 'Myanmar'),
    ('NA', 'Namibia'),
    ('NR', 'Nauru'),
    ('NP', 'Nepal'),
    ('NL', 'Netherlands'),
    ('AN', 'Netherlands Antilles'),
    ('NC', 'New Caledonia'),
    ('NZ', 'New Zealand'),
    ('NI', 'Nicaragua'),
    ('NE', 'Niger'),
    ('NG', 'Nigeria'),
    ('NU', 'Niue'),
    ('NF', 'Norfolk Island'),
    ('MP', 'Northern Mariana Islands'),
    ('NO', 'Norway'),
    ('OM', 'Oman'),
    ('PK', 'Pakistan'),
    ('PW', 'Palau'),
    ('PA', 'Panama'),
    ('PG', 'Papua New Guinea'),
    ('PY', 'Paraguay'),
    ('PE', 'Peru'),
    ('PH', 'Philippines'),
    ('PN', 'Pitcairn'),
    ('PL', 'Poland'),
    ('PT', 'Portugal'),
    ('PR', 'Puerto Rico'),
    ('QA', 'Qatar'),
    ('RE', 'Reunion'),
    ('RO', 'Romania'),
    ('RU', 'Russian Federation'),
    ('RW', 'Rwanda'),
    ('KN', 'Saint Kitts And Nevis'),
    ('LC', 'Saint Lucia'),
    ('VC', 'St Vincent/Grenadines'),
    ('WS', 'Samoa'),
    ('SM', 'San Marino'),
    ('ST', 'Sao Tome'),
    ('SA', 'Saudi Arabia'),
    ('SN', 'Senegal'),
    ('SC', 'Seychelles'),
    ('SL', 'Sierra Leone'),
    ('SG', 'Singapore'),
    ('SK', 'Slovakia'),
    ('SI', 'Slovenia'),
    ('SB', 'Solomon Islands'),
    ('SO', 'Somalia'),
    ('ZA', 'South Africa'),
    ('ES', 'Spain'),
    ('LK', 'Sri Lanka'),
    ('SH', 'St. Helena'),
    ('PM', 'St.Pierre'),
    ('SD', 'Sudan'),
    ('SR', 'Suriname'),
    ('SZ', 'Swaziland'),
    ('SE', 'Sweden'),
    ('CH', 'Switzerland'),
    ('SY', 'Syrian Arab Republic'),
    ('TW', 'Taiwan'),
    ('TJ', 'Tajikistan'),
    ('TZ', 'Tanzania'),
    ('TH', 'Thailand'),
    ('TG', 'Togo'),
    ('TK', 'Tokelau'),
    ('TO', 'Tonga'),
    ('TT', 'Trinidad And Tobago'),
    ('TN', 'Tunisia'),
    ('TR', 'Turkey'),
    ('TM', 'Turkmenistan'),
    ('TV', 'Tuvalu'),
    ('UG', 'Uganda'),
    ('UA', 'Ukraine'),
    ('AE', 'United Arab Emirates'),
    ('UK', 'United Kingdom'),
    ('UY', 'Uruguay'),
    ('UZ', 'Uzbekistan'),
    ('VU', 'Vanuatu'),
    ('VA', 'Vatican City State'),
    ('VE', 'Venezuela'),
    ('VN', 'Viet Nam'),
    ('VG', 'Virgin Islands (British)'),
    ('VI', 'Virgin Islands (U.S.)'),
    ('EH', 'Western Sahara'),
    ('YE', 'Yemen'),
    ('YU', 'Yugoslavia'),
    ('ZR', 'Zaire'),
    ('ZM', 'Zambia'),
    ('ZW', 'Zimbabwe')
]

In [175]:
len(Country)

231